In [1]:
%load_ext autoreload
%autoreload 2

In [6]:
import pandas as pd
from ecephys_analyses import load_shy as shy

In [7]:
def get_count(spws):
    return len(spws)

def get_rate(spws, hyp):
    count = get_count(spws)
    duration = hyp.duration.cumsum().max() / pd.to_timedelta(1, 's')
    return count / duration

In [8]:
fn = 'median'
states = [shy.NREM, shy.WAKE, shy.SPW_STATES]
state_names = ["NREM", "wake", "all-SPW-states"]

In [9]:
dfs = list()
for subject in shy.ALL_SUBJECTS:
    for subcondition in shy.get_subconditions():
        for state_name, state in zip(state_names, states):
            print(f"{subject}, {subcondition}, {state_name}")
            hyp = shy.load_hypnogram(subject, subcondition, keep_states=state)
            spws = shy.load_spws(subject, subcondition, hyp, abs_sink=True)
            incidence = pd.Series(
                {"count": get_count(spws), "rate": get_rate(spws, hyp)}
            )
            df = (
                spws.agg(fn, numeric_only=True)
                .append(incidence)
                .to_frame("value")
                .rename_axis("property")
                .assign(state=state_name, condition=subcondition, subject=subject)
                .reset_index()
            )
            dfs.append(df)
            print(df.to_markdown())

pd.concat(dfs).to_csv(f"{fn}-spw-properties.csv", index=False)

Segundo, recovery, NREM
|    | property       |         value | state   | condition   | subject   |
|---:|:---------------|--------------:|:--------|:------------|:----------|
|  0 | duration       |     0.0608    | NREM    | recovery    | Segundo   |
|  1 | sink_amplitude |     0.0696394 | NREM    | recovery    | Segundo   |
|  2 | sink_integral  |     2.84348   | NREM    | recovery    | Segundo   |
|  3 | count          | 11130         | NREM    | recovery    | Segundo   |
|  4 | rate           |     0.980965  | NREM    | recovery    | Segundo   |
Segundo, recovery, wake
|    | property       |        value | state   | condition   | subject   |
|---:|:---------------|-------------:|:--------|:------------|:----------|
|  0 | duration       |    0.0552    | wake    | recovery    | Segundo   |
|  1 | sink_amplitude |    0.0713913 | wake    | recovery    | Segundo   |
|  2 | sink_integral  |    2.62098   | wake    | recovery    | Segundo   |
|  3 | count          | 1970         | wake  